In [1]:
from config import active_config

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input, Dense, BatchNormalization, LSTM, GRU, Concatenate, RepeatVector, Embedding, Bidirectional, TimeDistributed
from keras.models import Model

C:\Users\windo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
class ImageCaptioningModel():
    
    def __init__(self, embedding_size=None, activation=None, regularizer=None, vocab_size=None, rnn_type=None, bidirectional_rnn=None, rnn_layers=None, rnn_output_size=None):
        self.embedding_size = embedding_size or active_config.embedding_size
        self.activation = activation or active_config.activation
        self.regularizer = regularizer or active_config.regularizer
        self.vocab_size = vocab_size or active_config.vocab_size
        self.rnn_type = rnn_type or active_config.rnn_type
        self.bidirectional_rnn = bidirectional_rnn or active_config.bidirectional_rnn
        self.rnn_layers = rnn_layers or active_config.rnn_layers
        self.rnn_output_size = rnn_output_size or active_config.rnn_output_size
        self.build()
        
    def build(self):
        
        image_input, image_embedding = self.get_image_embedding()
        word_input, word_embedding = self.get_word_embedding()
        
        seq_input = Concatenate(axis=1)(image_embedding, word_embedding)
        seq_output = self.build_sequence_model(seq_input)
        
        model = Model(inputs=[image_input, word_input], outputs=seq_output)
        
        self.keras_model = model
        
        #TODO Build model
        
    def get_image_embedding(self):
        #TODO Build model to calculate image embedding
        image_model = InceptionResNetV2(include_top=False, weights='imagenet')
        for layer in image_model.layers:
            layer.trainable = False
            
        dense_output = Dense(units=self.embedding_size, activation=self.activation, kernel_regularizer = self.regularizer)(image_model.output)
        
        image_embedding = RepeatVector(1)(dense_output)
        
        image_input = image_mode.input
        
        return image_input, image_embedding
        
    def get_word_embedding(self):
        #TODO Embed captions
        input_sentence = Input(shape=[None])
        
        embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, embeddings_regularizer=self.regularizer)(input_sentence)
        
        return input_sentence, embedding
        
    def build_sequence_model(self, seq_input):
        #TODO Build sequence model
        RNN = LSTM if self.rnn_type=='lstm' else GRU
        
        def rnn_layer():
            rnn = RNN(units=self.rnn_output_size)
            
            if self.bidirectional_rnn:
                rnn = Bidirectional(rnn)
            
            return rnn
        
        prev_input = seq_input
        for i in self.rnn_layers:
            prev_input = BatchNormalization()(prev_input)
            rnn_output = rnn_layer()(prev_input)
            prev_input = rnn_output
        time_distributed_dense_output = TimeDistributed(Dense(units=self.vocab_size))(rnn_output)
        return time_distributed_dense_output

In [7]:
model = ImageCaptioningModel()

ValueError: Input 0 is incompatible with layer repeat_vector_2: expected ndim=2, found ndim=4